# LAB 5: Higgs Classification

For this lab, we look at an inclusive search for the Standard Model's Higgs boson in high energy proton-proton collissions at $\sqrt{s} = 13 TEv$ at the Large Hadron Collider using simulated data. Higgs bosons are produced with large traverse momentum $(pT)$ and decay to a bottom quark - antiquark pair. We can use Calorimeters to reconstruct Higgs candinates as large-radius jets. Large QCD background containmination prevents us from achieving $5\sigma$ obervation of this Higgs channel. We'll be using a set of training datasets to optimize events selctreion in order to better the discovery sensitivity.

### Jets
Jets are collections of particles that go toward the same direction in the detector. They come form Bremsstrahlung(showering) and fragmentation of primordial hard quark or gluon. Jets must be defined from experimental observables, namely the 4-momenta of the observed particles in the event. One of the properties of jets is that they allow for calculation to be done at the parton level(quarks and gluons) to agree with a measurement done at the particle level(pions and protons). Another feature is that a jet algorithm should be local in the detector (not pulling in particles from far away). This reduces systematic uncertainty of jet measurements. This simplest way to define a jet is to draw a cone of size $R=\sqrt{(Δη)^2+Δ(ϕ)^2}$ around some particle and include everything in that cone as the jet.

In [1]:
# import library
import pickle

# open the file of interest, and use pickle loading
infile = open ("../higgs_100000_pt_1000_1200.pkl",'rb')
new_dict = pickle.load(infile)

# list all keys of the files
new_dict.keys()

Index(['pt', 'eta', 'phi', 'mass', 'ee2', 'ee3', 'd2', 'angularity', 't1',
       't2', 't3', 't21', 't32', 'KtDeltaR'],
      dtype='object')

In [2]:
# Print two variables, mass and d2, of the first 10 jets
for i in range(10):
 print(new_dict['mass'][i],new_dict['d2'][i])

126.39070494296645 1.565920565144541
125.73547207947769 0.8224079853829884
125.94694825108256 1.1595192989207739
133.47505458091868 1.1312895177410458
135.04731904755826 5.084334868566442
126.18831893079289 1.9893365548544923
132.22698108959537 0.8322677007024063
125.91413826896284 2.2895005833240227
125.00446344009822 0.8944416286773573
135.17626897095028 1.0176201623059749
